In [2]:
import numpy as np
import matplotlib.pyplot as plt
import loadParametersP1
import loadFittingDataP1
from scipy.stats import multivariate_normal
import pdb
import math
import copy

In [3]:
class GD(object):
    
    def __init__(self, x0, objective,
                 gradient=None,
                 step_size=0.1):
        self.x0 = x0
        self.objective = objective
        self.gradient = gradient
        self.step_size = step_size
    
    def compute_gradient(self, x, idx=None, eps=1e-6):
        if self.gradient != None:
            return self.gradient(x)
        grad = np.array([0.0 for i in range(len(x))])
        if idx != None:
            f_x = self.objective(x)
            x[idx] += eps
            f_eps = self.objective(x)
            x[idx] -= eps
            grad[idx] = (f_eps-f_x)/eps
            return grad
        X = copy.copy(x)
        for i in range(len(x)):
            f_x = self.objective(X)
            #print X, X[i]+eps, self.objective(X), eps
            X[i] = X[i] + eps
            #print X, X[i], self.objective(X)
            f_eps = self.objective(X)
            X[i] = X[i] - eps
            grad[i] = (f_eps-f_x)/eps
        return grad
        
    
    def step(self, stochastic=False, gtol=1e-8):
        log = []
        while True:
            grad = self.compute_gradient(self.x0)
            log.append((self.x0, self.objective(self.x0)))
            if np.linalg.norm(grad) < gtol:
                break
            self.x0 = self.x0 - self.step_size * grad
        return log

In [4]:
gaussMean,gaussCov,quadBowlA,quadBowlb = loadParametersP1.getData()

In [5]:
test_gauss = lambda x : -multivariate_normal.pdf(x, gaussMean, gaussCov)
test_gauss_gradient = lambda x : -test_gauss(x)*np.linalg.inv(gaussCov).dot(x-gaussMean)

In [6]:
test_bowl = lambda x : 0.5*x.T.dot(quadBowlA.dot(x)) - x.T.dot(quadBowlb)
test_bowl_gradient = lambda x : quadBowlA.dot(x) - quadBowlb

In [7]:
x0 = np.array([6.0, 16.0])
log = GD(x0, test_gauss, None, 10000).step()

In [8]:
log[-1], len(log)

((array([  9.9651509 ,  10.05227234]), -0.00015915462901127802), 2987)

In [9]:
X,y = loadFittingDataP1.getData()

In [150]:
class SGD(object):
    
    def __init__(self, X, y,
                 step_size=1e-5):
        self.X = X
        self.y = y
        self.step_size = step_size
    
    def compute_objective(self, theta):
        return sum([(theta.dot(X[i])-y[i])**2 for i in range(len(y))])
    
    def compute_numerical_gradient(self, theta, eps=1e-7, idx=None):
        grad = np.zeros(len(theta))
        for i in range(len(theta)):
            if idx != None and i != idx:
                continue
            f = self.compute_objective(theta)
            theta[i] = theta[i] + eps
            f_eps = self.compute_objective(theta)
            theta[i] = theta[i] - eps
            grad[i] = (f_eps-f)/eps
        return grad
        
    def compute_gradient(self, theta, idx=None):
        if idx == None:
            idx = range(len(self.y))
        grad = np.zeros(self.X.shape[1])
        for i in idx:
            grad += 2 * (self.X[i].dot(theta) - self.y[i]) * self.X[i]
        return grad
    
    def step(self, theta, stochastic=False, minibatch_size=1, ftol=1e-7):
        log = []
        idx = None
        if stochastic:
            idx = np.random.randint(self.X.shape[1], size=minibatch_size)
        prev_objective = self.compute_objective(theta)
        t_0 = 1.0
        t = 0
        k = 0.95
        while True:
            grad = self.compute_gradient(theta, idx)
            log.append((theta, prev_objective))
            step_size = 1e-4*math.pow(t_0+t, -k)
            theta = theta - step_size * grad
            tmp = self.compute_objective(theta)
            if abs(tmp-prev_objective) < ftol:
                break
            prev_objective = tmp
            t += 1
            if t%10 == 0:
                print log[-1][1]
            if t > 10004: break
        return log

In [151]:
optimizer = SGD(X,y, 1e-5)

In [152]:
theta_0 = np.random.random(X.shape[1])
print optimizer.compute_gradient(theta_0) - optimizer.compute_numerical_gradient(theta_0)

[-0.12505523 -0.09585159 -0.0785605  -0.06884348 -0.08581199  0.03567473
 -0.0645412  -0.04358974  0.06707632  0.09405961]


In [153]:
log = optimizer.step(theta_0, stochastic=True, minibatch_size=100);

2.24609159552e+21
9.4299003205e+23
7.15041872526e+15
2623390.4435
2527161.15091
2452048.74019
2390807.70452
2339327.83404
2295064.03819
2256336.56225
2221981.52125
2191160.93908
2163252.46255
2137781.8652
2114379.91303
2092753.78422
2072667.54221
2053928.43744
2036377.07523
2019880.2149
2004325.40135
1989616.89868
1975672.56594
1962421.42565
1949801.74933
1937759.53406
1926247.2783
1915222.9895
1904649.37289
1894493.16329
1884724.57097
1875316.81891
1866245.75416
1857489.5195
1849028.27452
1840843.95756
1832920.08159
1825241.55828
1817794.54585
1810566.31696
1803545.14346
1796720.19561
1790081.45365
1783619.6298
1777326.09948
1771192.84034
1765212.37805
1759377.73803
1753682.40241
1748120.27141
1742685.62872
1737373.11039
1732177.67676
1727094.58712
1722119.3768
1717247.83637
1712475.99278
1707800.09214
1703216.58413
1698722.10766
1694313.47783
1689987.67394
1685741.82856
1681573.21739
1677479.25005
1673457.4615
1669505.5042
1665621.14084
1661802.2376
1658046.75795
1654352.75689
165071

In [154]:
log[10000][1]

1086039.7426225974

In [21]:
X_dagger = np.linalg.inv(X.T.dot(X)).dot(X.T)
optimizer.compute_objective(X_dagger.dot(y))

8333.2142111780468

In [139]:
theta_0 = np.random.random(X.shape[1])